### single string

In [19]:
import os
import requests
from dotenv import load_dotenv 

load_dotenv()



api_key  = os.getenv('API_KEY')


headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + api_key,
}

json_data = {
    'input': 'What does the chamber of commerce do?',
    'model': 'text-embedding-ada-002',
}

response = requests.post('https://api.openai.com/v1/embeddings', headers=headers, json=json_data)
query = response.json()['data'][0]['embedding'] #len=1536 #pricing=0.0004

### multiple strings

In [20]:
import os
import requests

headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + api_key,
}

json_data = {
    'input': ['Your text string goes here', 'I hate you'],
    'model': 'text-embedding-ada-002',
}

response = requests.post('https://api.openai.com/v1/embeddings', headers=headers, json=json_data)
response.json()['data']

dict1 = dict()
for index in range(len(json_data['input'])):
    dict1[json_data['input'][index]] = response.json()['data'][index]['embedding']
dict1

{'Your text string goes here': [-0.0069292835,
  -0.005336422,
  0.011875892,
  -0.025001878,
  -0.024692714,
  0.039787933,
  -0.010101565,
  -0.009402588,
  -0.013186474,
  -0.009940263,
  -0.011680985,
  0.007870215,
  -0.014087079,
  0.0077694007,
  0.010168774,
  -0.00503398,
  0.022945272,
  -0.0015122104,
  0.014933917,
  -0.010262867,
  0.0048625963,
  0.012460613,
  0.004909643,
  0.010807263,
  -0.0066033183,
  -0.00035746972,
  0.005578376,
  -0.012494218,
  0.016372196,
  0.00452991,
  0.006562993,
  -0.007070423,
  -0.015216196,
  -0.0065898765,
  -0.018724523,
  0.0040796073,
  0.0032126068,
  -0.01899336,
  0.030163554,
  -0.007594656,
  0.008233145,
  0.00942275,
  -0.0010887914,
  -0.00040955696,
  -0.008831308,
  -0.028631182,
  0.0030176996,
  0.009295053,
  0.01654694,
  0.0064756204,
  0.019436942,
  0.014718847,
  -0.025754621,
  -0.009913378,
  -0.0033403046,
  0.014907033,
  -0.036911372,
  0.013912335,
  0.013408265,
  0.00438541,
  -0.0048357127,
  0.003226048

In [21]:
def chatgpt3_question(context, question):
    url = "https://api.openai.com/v1/chat/completions"

    prompt = f"""
    based on this context: {context}
    answer this use question: {question}
    """

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    data = {
        "model": "gpt-3.5-turbo",
        "messages": [{"role": "user", "content": prompt}]
    }

    response = requests.post(url, headers=headers, json=data)
    generated_text = response.json()['choices'][0]['message']['content']

    return generated_text

### in-context based chatbot

In [22]:
import os
import requests
import pandas as pd

def split_paragraph(text, keyword):
    list1 = [x.strip() for x in text.split('.')]
    list2 = []
    
    for sentence in list1:
        # Check if the sentence contains the phrase "chamber of commerce"
        if keyword in sentence.lower():
            list2.append(1)
        else:
            list2.append(0)

    #in case first sentence has no keyword, we add it
    if list2[0] == 0:
        list1[0] = f'the {keyword}: ' + list1[0]
        list2[0] = 1

    # print(list1)
    # print(list2)

    list3 = list()
    current_string = ''
    # Loop through each element of list1 and list2
    for i in range(len(list1)):
        # If the corresponding element in list2 is 1, add the current string to list3 and reset the current string

        if list2[i] == 1:
            list3.append(current_string)
            current_string = "" #reset
            current_string += list1[i]

        # Otherwise, concatenate the current string with the current element of list1
        if list2[i] == 0:
            current_string += '. '+list1[i]

    # Add the final concatenated string to list3
    list3.append(current_string)

    return [x.strip() for x in list3[1:]]

def context_dict2context_list(context_dict):
    list1 = list()
    for key in context_dict:
        # print(key)
        str1 = context_dict[key]
        
        split_list = [x.replace('\n', '').strip() for x in str1.split('\n\n')]
        split_list

        for sentence in split_list:
            for s in split_paragraph(sentence, key):
                list1.append(s)
    return list1

def list2vec(list1):
    headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer ' + api_key,
    }

    json_data = {
        'input': list1,
        'model': 'text-embedding-ada-002',
    }

    response = requests.post('https://api.openai.com/v1/embeddings', headers=headers, json=json_data)
    return [x['embedding'] for x in response.json()['data']]

    dict1 = dict()
    for index in range(len(json_data['input'])):
        dict1[json_data['input'][index]] = response.json()['data'][index]['embedding']
    return dict1

def text2vec(query):
    headers = {
        'Content-Type': 'application/json',
        'Authorization': 'Bearer ' + api_key,
    }

    json_data = {
        'input': query,
        'model': 'text-embedding-ada-002',
    }

    response = requests.post('https://api.openai.com/v1/embeddings', headers=headers, json=json_data)
    query = response.json()['data'][0]['embedding'] #len=1536 #pricing=0.0004
    return query

In [23]:
context_dict = {
    "chamber of commerce" : 
    """
    The chamber of commerce is an organization that represents businesses and industries within a specific geographical area. Its main purpose is to promote economic growth, trade, and investment in the region it serves. 
    
    One of the key activities of the chamber of commerce is advocacy. It lobbies government officials and policymakers on behalf of its members, advocating for policies and regulations that support a favorable business environment. This includes advocating for tax breaks, infrastructure improvements, and other measures that can help businesses thrive. Additionally, the chamber of commerce may work with other organizations to promote a particular industry or sector, such as tourism or technology.

    Another important activity of the chamber of commerce is networking. It provides opportunities for its members to connect with one another, build relationships, and exchange ideas. This may include hosting networking events, conferences, and seminars. By facilitating connections between businesses, the chamber of commerce can help foster collaboration and innovation within the local economy.

    The chamber of commerce also provides a range of services to its members, including training and education, access to resources and information, and marketing and advertising support. For example, it may offer workshops on topics such as social media marketing, or provide members with access to databases and other resources that can help them grow their businesses. Additionally, the chamber of commerce may promote its members through advertising and other marketing initiatives, helping to raise their visibility and attract new customers.

    Overall, the chamber of commerce plays a vital role in supporting economic growth and development in the communities it serves. Through its advocacy, networking, and service activities, it helps to create a favorable business environment, foster innovation and collaboration, and provide valuable resources and support to its members.

    Overall, the chamber of commerce plays a vital role in supporting economic growth and development in the communities it serves. Through its advocacy, networking, and service activities, it helps to create a favorable business environment, foster innovation and collaboration, and provide valuable resources and support to its members.

    The Italian Chamber of Hong Kong and Macao, in collaboration with 10 International Business Chambers, is holding a monthly Joint-Chambers Happy Hour on Wednesday, April 12, 2023, from 6:00 pm to 9:00 pm at I-O-N Central Market​.

    The Italian Chamber of Commerce is sponsoring a networking event: GBA & HAINAN - Opportunities for Italian companies, on Thursday, April 13, from 2:30 pm to 3:30 pm. The event will be held at the Hainan International Convention and Exhibition Center and Online.
    """
    ,
    "price" :
    """
    Our price are about 50 USD/hour per accounting consultation. If the consultation is about law, then 70 USD/hour price. The price of booking a face to face meeting is 100 USD/hour.
    """,
    "Membership":
    """
    Members of the organization can be found here: https://iccj.or.jp/membership/
    Benefits of membership can be found here: https://iccj.or.jp/why-join/
    Example Benefits: 15 to 35 percent discount on ICCJ services, Exclusive access to events or limited access areas/lounge on the occasion of major ICCJ events, etc.
    """
    ,
    "Contact":
    """
    If you would like to contact the Italian Chamber of Commerce you can visit https://iccj.or.jp/contact-info/. There you will find contact email, phone number, fax, and business hours.

    Business Hours is between 9:30-18:00 Japanese time.
    """
    ,
    "Location":
    """
    The Italian Chamber of Commerce is located 10 minutes walk from Mita, Shirokane-Takanawa or Tamachi Station. https://iccj.or.jp/access/ provides all the information about how to visit us.
    """,
    "Techinical Support":
    """
    In case someone asks for help on technical issue, say yes
    """
}

import pandas as pd
from sentence_transformers import SentenceTransformer, util

#prepare context
context_list = context_dict2context_list(context_dict)
context_list

#create df
df = pd.DataFrame([context_list, list2vec(context_list)]
).T
df.columns = ['description', 'text_vector_']
df['description'] = df['description'].apply(lambda x : x.strip())
df

,description,text_vector_
0,The chamber of commerce is an organization tha...,"[-0.015116395, -0.0055863564, 0.028409708, -0...."
1,One of the key activities of the chamber of co...,"[-0.0054941755, -0.0047648316, -0.0023775965, ..."
2,"Additionally, the chamber of commerce may work...","[-0.0018640445, -0.0062578637, 0.00772247, -0...."
3,Another important activity of the chamber of c...,"[-0.018242562, -0.009165559, 0.027806625, -0.0..."
4,By facilitating connections between businesses...,"[-0.013676331, -0.0045832293, 0.0051062983, -0..."
5,The chamber of commerce also provides a range ...,"[-0.017254777, -0.005237597, 0.004493968, -0.0..."
6,"Additionally, the chamber of commerce may prom...","[-0.015887452, -0.0014834765, 0.008638085, -0...."
7,"Overall, the chamber of commerce plays a vital...","[-0.015497115, 0.005012449, 0.0103142625, -0.0..."
8,"Overall, the chamber of commerce plays a vital...","[-0.015497115, 0.005012449, 0.0103142625, -0.0..."
9,the chamber of commerce: The Italian Chamber o...,"[0.0029192464, -0.017682098, 0.010473243, -0.0..."


In [24]:
qa_list = {
    'for how many years have you been operating?' : '20',
    'how big is your team?' : '10 people',
    'how many people are working for the chamber?' : '10 people'
}
df_qa = pd.DataFrame([qa_list]).T.reset_index()
df_qa.columns = ['question', 'answer']
df_qa['text_vector_'] = list2vec(df_qa['question'].values.tolist())
df_qa

,question,answer,text_vector_
0,for how many years have you been operating?,20,"[0.010152486, -0.022128223, 0.010703396, -0.04..."
1,how big is your team?,10 people,"[0.010457992, -0.006170015, 0.023638932, -0.00..."
2,how many people are working for the chamber?,10 people,"[-0.008994513, 0.00088858436, -0.0030945807, -..."


In [25]:
df_qa_ = df_qa.copy()
df_ = df.copy()

def qa(df_, df_qa_, min_qa_score, min_context_score, verbose, query):
    query_vec = text2vec(query)

    #first check if there is already a question in df_qa
    df_qa_['score'] = df_qa_['text_vector_'].apply(lambda x : float(util.cos_sim(x, query_vec)))
    df_qa_ = df_qa_.sort_values('score', ascending=False)
    df_qa_ = df_qa_[df_qa_['score']>=min_qa_score]
    #if we find at least one possible preset answer
    if len(df_qa_) > 0:
        if verbose : display(df_qa_)
        answer = df_qa_[0:1]['answer'].values.tolist()[0]
        return answer
    
    #then check if we can use the context to answer a question
    df_['score'] = df_['text_vector_'].apply(lambda x : float(util.cos_sim(x, query_vec)))
    df_ = df_.sort_values('score', ascending=False)
    df_ = df_[df_['score']>=min_context_score]
    #if we find at least one possible preset answer
    if len(df_) > 0:
        if verbose : display(df_)
        #in case we might decide to merge multiple context
        context = ' '.join(df_['description'][0:1].values.tolist())
        answer = chatgpt3_question(context, query)
        return answer
    else:
        return 'impossible to give an answer'

print(
    qa(
        df_, 
        df_qa_, 
        min_qa_score=0.92, 
        min_context_score=.75, 
        verbose=True, 
        query='What does the chamber of commerce do?'
    )
)

,description,text_vector_,score
6,"Additionally, the chamber of commerce may prom...","[-0.015887452, -0.0014834765, 0.008638085, -0....",0.922444
0,The chamber of commerce is an organization tha...,"[-0.015116395, -0.0055863564, 0.028409708, -0....",0.913271
2,"Additionally, the chamber of commerce may work...","[-0.0018640445, -0.0062578637, 0.00772247, -0....",0.913127
7,"Overall, the chamber of commerce plays a vital...","[-0.015497115, 0.005012449, 0.0103142625, -0.0...",0.912536
8,"Overall, the chamber of commerce plays a vital...","[-0.015497115, 0.005012449, 0.0103142625, -0.0...",0.912536
5,The chamber of commerce also provides a range ...,"[-0.017254777, -0.005237597, 0.004493968, -0.0...",0.910604
1,One of the key activities of the chamber of co...,"[-0.0054941755, -0.0047648316, -0.0023775965, ...",0.909669
3,Another important activity of the chamber of c...,"[-0.018242562, -0.009165559, 0.027806625, -0.0...",0.901163
4,By facilitating connections between businesses...,"[-0.013676331, -0.0045832293, 0.0051062983, -0...",0.897961
9,the chamber of commerce: The Italian Chamber o...,"[0.0029192464, -0.017682098, 0.010473243, -0.0...",0.829980


The chamber of commerce promotes its members through advertising and other marketing initiatives, helping to raise their visibility and attract new customers.


In [26]:
questions = [    "Can you provide information about your products/services?",
               "Can you tell me about your pricing plans?",    "What are your business hours?", 
                     "Can you help me schedule an appointment?", 
                               "Can you provide assistance with technical issues?",  
                                   "How can I contact customer support?",  
                                       "Do you offer any discounts or promotions?",   
                                       "What is your company's history and mission statement?"]
ans = []

In [27]:
for question in questions:
    temp = qa(
            df_, 
            df_qa_, 
            min_qa_score=0.92, 
            min_context_score=.75, 
            verbose=True, 
            query=question
        )
    ans.append(temp)

,description,text_vector_,score
18,the Techinical Support: In case someone asks f...,"[0.0065836944, -0.014227586, 0.013730209, -0.0...",0.787590
5,The chamber of commerce also provides a range ...,"[-0.017254777, -0.005237597, 0.004493968, -0.0...",0.780928
16,the Contact: Business Hours is between 9:30-18...,"[-0.017350322, -0.008873829, 0.0023459357, -0....",0.773903
11,Our price are about 50 USD/hour per accounting...,"[0.016638244, 0.0064203353, -0.0075359135, -0....",0.772331
6,"Additionally, the chamber of commerce may prom...","[-0.015887452, -0.0014834765, 0.008638085, -0....",0.771071
4,By facilitating connections between businesses...,"[-0.013676331, -0.0045832293, 0.0051062983, -0...",0.765521
2,"Additionally, the chamber of commerce may work...","[-0.0018640445, -0.0062578637, 0.00772247, -0....",0.763666
15,the Contact: If you would like to contact the ...,"[0.0014398829, 0.010223489, 0.022448063, -0.04...",0.762888
10,The Italian Chamber of Commerce is sponsoring ...,"[-0.023526464, -0.029451745, 0.022585941, -0.0...",0.759286
3,Another important activity of the chamber of c...,"[-0.018242562, -0.009165559, 0.027806625, -0.0...",0.755726


,description,text_vector_,score
11,Our price are about 50 USD/hour per accounting...,"[0.016638244, 0.0064203353, -0.0075359135, -0....",0.804098
13,The price of booking a face to face meeting is...,"[0.0007829916, -0.0071963, -0.0039067506, -0.0...",0.780992
12,"If the consultation is about law, then 70 USD/...","[0.023638053, 0.006782695, 0.0030937463, -0.02...",0.763024
18,the Techinical Support: In case someone asks f...,"[0.0065836944, -0.014227586, 0.013730209, -0.0...",0.750314


,description,text_vector_,score
16,the Contact: Business Hours is between 9:30-18...,"[-0.017350322, -0.008873829, 0.0023459357, -0....",0.863785
11,Our price are about 50 USD/hour per accounting...,"[0.016638244, 0.0064203353, -0.0075359135, -0....",0.786034
15,the Contact: If you would like to contact the ...,"[0.0014398829, 0.010223489, 0.022448063, -0.04...",0.781597
9,the chamber of commerce: The Italian Chamber o...,"[0.0029192464, -0.017682098, 0.010473243, -0.0...",0.776148
13,The price of booking a face to face meeting is...,"[0.0007829916, -0.0071963, -0.0039067506, -0.0...",0.773112
10,The Italian Chamber of Commerce is sponsoring ...,"[-0.023526464, -0.029451745, 0.022585941, -0.0...",0.769523
17,the Location: The Italian Chamber of Commerce ...,"[-0.0043017813, 0.0012815585, -0.0091687115, -...",0.755749
4,By facilitating connections between businesses...,"[-0.013676331, -0.0045832293, 0.0051062983, -0...",0.753887
12,"If the consultation is about law, then 70 USD/...","[0.023638053, 0.006782695, 0.0030937463, -0.02...",0.752972
5,The chamber of commerce also provides a range ...,"[-0.017254777, -0.005237597, 0.004493968, -0.0...",0.751923


,description,text_vector_,score
13,The price of booking a face to face meeting is...,"[0.0007829916, -0.0071963, -0.0039067506, -0.0...",0.777118
16,the Contact: Business Hours is between 9:30-18...,"[-0.017350322, -0.008873829, 0.0023459357, -0....",0.766100
18,the Techinical Support: In case someone asks f...,"[0.0065836944, -0.014227586, 0.013730209, -0.0...",0.761876
11,Our price are about 50 USD/hour per accounting...,"[0.016638244, 0.0064203353, -0.0075359135, -0....",0.752297


,description,text_vector_,score
18,the Techinical Support: In case someone asks f...,"[0.0065836944, -0.014227586, 0.013730209, -0.0...",0.866027


,description,text_vector_,score
18,the Techinical Support: In case someone asks f...,"[0.0065836944, -0.014227586, 0.013730209, -0.0...",0.805974
15,the Contact: If you would like to contact the ...,"[0.0014398829, 0.010223489, 0.022448063, -0.04...",0.794465
16,the Contact: Business Hours is between 9:30-18...,"[-0.017350322, -0.008873829, 0.0023459357, -0....",0.782605


,description,text_vector_,score
11,Our price are about 50 USD/hour per accounting...,"[0.016638244, 0.0064203353, -0.0075359135, -0....",0.771326
6,"Additionally, the chamber of commerce may prom...","[-0.015887452, -0.0014834765, 0.008638085, -0....",0.765951
13,The price of booking a face to face meeting is...,"[0.0007829916, -0.0071963, -0.0039067506, -0.0...",0.753573
2,"Additionally, the chamber of commerce may work...","[-0.0018640445, -0.0062578637, 0.00772247, -0....",0.753208
14,the Membership: Members of the organization ca...,"[-0.01318125, 0.00637876, 0.031264726, -0.0434...",0.751100


,description,text_vector_,score
6,"Additionally, the chamber of commerce may prom...","[-0.015887452, -0.0014834765, 0.008638085, -0....",0.755411
0,The chamber of commerce is an organization tha...,"[-0.015116395, -0.0055863564, 0.028409708, -0....",0.753757
4,By facilitating connections between businesses...,"[-0.013676331, -0.0045832293, 0.0051062983, -0...",0.751186


In [28]:
ans

['I am sorry, as an AI language model, I do not have any products or services to provide information about. Can you please provide more context about the products or services you are interested in?',
 'Our pricing for accounting consultations is approximately 50 USD per hour.',
 'Our business hours are between 9:30-18:00 Japanese time.',
 'As an AI language model, I am not capable of scheduling appointments. However, based on the context given, the price for booking a face to face meeting is 100 USD per hour. If you are interested in scheduling an appointment, you may need to contact the relevant person or organization to make the necessary arrangements.',
 'Yes.',
 "Unfortunately, as an AI language model, I do not have specific information on how to contact customer support. However, it is advisable to check the company's website, social media pages or product documentation for contact information. You can also try calling the company's customer support hotline or sending them an emai